In [67]:
## Import packages
import os
import mne
import numpy as np
import scipy as sp
import scipy.io as sio
import matplotlib.pyplot as plt

from scipy.signal import butter, sosfiltfilt, sosfreqz  
from scipy.io import loadmat
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, roc_curve, auc

In [68]:
## Define directories
data_dir = '/home/inffzy/Desktop/cogs189/cogs189_final_project/data'

In [69]:
## Raw data directory names
bc4_2a_processed_name = 'bci_competition_4_2a_processed'
bc3_3a_processed_name = 'bci_competition_3_3a_processed'

## Create list of all data paths
bc4_2a_processed_data_paths = []
bc3_3a_processed_data_paths = []

## Add bc4_2a data paths
for subject_idx in range(1, 10):
    subject_name = 'A0' + str(subject_idx) + 'T'
    bc4_2a_processed_data_paths.append(
        os.path.join(data_dir, bc4_2a_processed_name, subject_name + '.npz'))
    
## Add bc3_3a data paths
for subject_idx in range(1, 4):
    subject_name = 'bc3_3a_s' + str(subject_idx)
    bc3_3a_processed_data_paths.append(
        os.path.join(data_dir, bc3_3a_processed_name, subject_name + '.npz'))

In [70]:
bc4_2a_s1_data_npz = np.load(bc4_2a_processed_data_paths[0])
bc4_2a_s1_data = bc4_2a_s1_data_npz['processed_motor_imagery_data']
bc4_2a_s1_labels = bc4_2a_s1_data_npz['descriptions']

In [72]:
bc4_2a_s1_data_ds = windowed_means(bc4_2a_s1_data, 100)

In [73]:
bc4_2a_s1_data_ds.shape

(273, 22, 100)

In [79]:
## Prepare training and testing data
X_train, X_test, y_train, y_test = train_test_split(bc4_2a_s1_data_flattened, 
                                                    bc4_2a_s1_labels, 
                                                    test_size=0.33, 
                                                    shuffle=True,
                                                    random_state=42)

In [80]:
X_train.shape

(182, 2200)

In [32]:
final_pred = one_vs_one_classifier(X_train, y_train)
#final_pred = one_vs_one_classifier(X_train, y_train, X_test, y_test, cross_val=5)

Cross validation scores for labels  769  and  770 :  [0.6        0.7        0.6        0.35       0.57894737]
Cross validation scores for labels  769  and  771 :  [0.42105263 0.61111111 0.44444444 0.5        0.44444444]
Cross validation scores for labels  769  and  772 :  [0.55555556 0.38888889 0.61111111 0.52941176 0.52941176]
Cross validation scores for labels  770  and  771 :  [0.68421053 0.57894737 0.57894737 0.52631579 0.55555556]
Cross validation scores for labels  770  and  772 :  [0.63157895 0.66666667 0.66666667 0.61111111 0.44444444]
Cross validation scores for labels  771  and  772 :  [0.47058824 0.29411765 0.47058824 0.4375     0.375     ]


In [35]:
print(accuracy_score(y_train, final_pred))
#print(accuracy_score(y_test, final_pred))

1.0


In [65]:
#final_pred = one_vs_one_CSP_classifier(X_train, y_train)
final_pred = one_vs_one_CSP_classifier(X_train, y_train, X_test, y_test, cross_val=5)

(48, 22, 750)
(51, 22, 750)
(99, 4500)
Mean cross validation score for labels  769  and  770 :  0.5552631578947368
(48, 22, 750)
(43, 22, 750)
(91, 4500)
Mean cross validation score for labels  769  and  771 :  0.6929824561403508
(48, 22, 750)
(40, 22, 750)
(88, 4500)
Mean cross validation score for labels  769  and  772 :  0.5352941176470588
(51, 22, 750)
(43, 22, 750)
(94, 4500)
Mean cross validation score for labels  770  and  771 :  0.6286549707602339
(51, 22, 750)
(40, 22, 750)
(91, 4500)
Mean cross validation score for labels  770  and  772 :  0.6046783625730995
(43, 22, 750)
(40, 22, 750)
(83, 4500)
Mean cross validation score for labels  771  and  772 :  0.5279411764705882


In [66]:
#print(accuracy_score(y_train, final_pred))
print(accuracy_score(y_test, final_pred))

0.31868131868131866


In [81]:
## Test LDA
lda = LinearDiscriminantAnalysis(solver='lsqr',  shrinkage='auto')
#lda.fit(X_train, y_train)

# Let's do 5-fold cross validation
#score_lda = cross_val_score(lda.fit(X_train, y_train), X_train, y_train, cv = 5)

# We will print out the mean score
#print("solver = lsqr  accuracy: %f" % np.mean(score_lda))

#y_pred = lda.predict(X_test)
#print(accuracy_score(y_test, y_pred))


one_vs_one = OneVsOneClassifier(lda).fit(X_train, y_train)
#one_vs_one_cross_score = cross_val_score(one_vs_one, X_train, y_train, cv = 5)

#print(one_vs_one_cross_score)

y_pred = one_vs_one.predict(X_test)
print(y_pred)
print(accuracy_score(y_test, y_pred))


[0.16216216 0.2972973  0.16666667 0.33333333 0.22222222]
[769 771 770 770 772 770 770 769 772 769 770 772 769 770 770 771 770 772
 770 770 769 771 772 769 769 771 769 771 770 772 771 769 770 771 769 772
 770 772 772 771 770 770 771 770 772 769 770 771 769 769 770 771 770 770
 770 772 770 770 770 772 769 772 769 769 770 772 771 769 772 769 770 770
 770 770 771 770 769 769 772 770 772 769 772 770 769 769 769 770 770 770
 771]
0.2857142857142857


In [ ]:
num_bc4_2a_channels = bc4_2a_s1_data.shape[1]

In [39]:
bc4_2a_s1_data.shape

(273, 22, 750)

In [40]:
## Separate subject data by trial label
bc4_2a_s1_data_c1 = bc4_2a_s1_data[bc4_2a_s1_labels == 769]
bc4_2a_s1_data_c2 = bc4_2a_s1_data[bc4_2a_s1_labels == 770]
bc4_2a_s1_data_c3 = bc4_2a_s1_data[bc4_2a_s1_labels == 771]
bc4_2a_s1_data_c4 = bc4_2a_s1_data[bc4_2a_s1_labels == 772]

num_bc4_2a_s1_c1_trials = bc4_2a_s1_data_c1.shape[0]
num_bc4_2a_s1_c2_trials = bc4_2a_s1_data_c2.shape[0]
num_bc4_2a_s1_c3_trials = bc4_2a_s1_data_c3.shape[0]
num_bc4_2a_s1_c4_trials = bc4_2a_s1_data_c4.shape[0]

In [46]:
W12 = CSP(bc4_2a_s1_data_c1, bc4_2a_s1_data_c2, n_top=3, n_bot=3)

In [47]:
W12.shape

(6, 22)

In [ ]:
def SACSP_Eq_4_or_5(data_fourier, h_or_l, w_or_v, avg_cov_sum):
    
    numerator = (w_or_v.T @ data_fourier @ np.diag(h_or_l) @ np.matrix(data_fourier).H @ w_or_v)[0, 0]
    denominator = (w_or_v.T @ avg_cov_sum @ w_or_v)[0, 0]
    
    return numerator / denominator

In [ ]:
def SACSP_Eq_6_or_7(data_fourier, h_or_l, avg_cov_sum, top_n):
    E = data_fourier @ np.diag(h_or_l) @ np.matrix(data_fourier).H
    
    ## Generalized eigen-decomposition of Eq6 and Eq7 along with sum of class' spatial covariances
    eigval, eigvec = sp.linalg.eigh(E, avg_cov_sum)
    
    ## Extract top n eigen vectors
    sort_indices = np.argsort(eigval)
    top_n_indices = list(sort_indices[-top_n:])
    top_n_eigvec = eigvec[:, top_n_indices]
    
    return top_n_eigvec

In [ ]:
def SACSP(c1_data, c2_data, R=3, M=1, e=1e-6):
    
    ## R = number of spectral/spatial filters for each class
    ## M = number of initializations of spectral filters
    
    t = c1_data.shape[-1]  ## Number of time samples
    F = sp.linalg.dft(t)  ## Fourier matrix with shape t x t
    
    H = np.ones((M, t))  ## Initialize M number of h vectors
    L = np.ones((M, t))  ## Initialize M number of l vectors
 
    c1_data_avg = np.mean(c1_data, axis=0)
    c2_data_avg = np.mean(c2_data, axis=0)

    c1_data_avg_cov = np.cov(c1_data_avg)
    c2_data_avg_cov = np.cov(c2_data_avg)
    avg_cov_sum = c1_data_avg_cov + c2_data_avg_cov
    
    c1_data_avg_fourier = np.fft.fft(c1_data_avg)
    c2_data_avg_fourier = np.fft.fft(c2_data_avg)
    
    c1_filter_pairs = []
    c2_filter_pairs = []
    
    for m in range(M):
        
        ## h and l are spectral filters
        h = H[m, :]
        l = L[m, :]
        
        ## w and v are spatial filters
        W = SACSP_Eq_6_or_7(c1_data_avg_fourier, h, avg_cov_sum, top_n=R)
        V = SACSP_Eq_6_or_7(c2_data_avg_fourier, l, avg_cov_sum, top_n=R)
        
        for r in range(R):  
            
            ### Class 1 optimization ###
            
            w = np.atleast_2d(W[:, r]).T  ## From W, get w as a column vector
            
            c1_cost = 0
            c1_cost_increase = e + 1
            
            while c1_cost_increase > e:
            
                ## Update spectral filter h_r_m from Eq9    
                E = np.matrix(c1_data_avg_fourier).H @ w @ w.T @ c1_data_avg_fourier
                
                for k in range(t):
                    h[k] = E[k, k] / np.linalg.norm(np.diag(E))
                
                H[m, :] = h
                
                ## Update spatial filter w_r by selecting the eigenvector 
                ##   corresponding to the largest eigenvalue from Eq6
                w = SACSP_Eq_6_or_7(c1_data_avg_fourier, h, avg_cov_sum, top_n=1)
                W[:, r] = w.flatten()

                ## Calculate cost
                c1_cost_prev = c1_cost
                c1_cost = SACSP_Eq_4_or_5(c1_data_avg_fourier, h, w, avg_cov_sum)
                c1_cost_increase = c1_cost - c1_cost_prev
   
            c1_filter_pairs.append([h, w])
            
            
            ### Class 2 optimization ###
            
            v = np.atleast_2d(V[:, r]).T  ## From V, get v as a column vector
            
            c2_cost = 0
            c2_cost_increase = e + 1
            
            while c2_cost_increase > e:
            
                ## Update spectral filter l_r_m from Eq10    
                E = np.matrix(c2_data_avg_fourier).H @ v @ v.T @ c2_data_avg_fourier
                
                for k in range(t):
                    l[k] = E[k, k] / np.linalg.norm(np.diag(E))
                
                L[m, :] = l
                
                ## Update spatial filter v_r by selecting the eigenvector 
                ##   corresponding to the largest eigenvalue from Eq7
                v = SACSP_Eq_6_or_7(c2_data_avg_fourier, l, avg_cov_sum, top_n=1)
                V[:, r] = v.flatten()

                ## Calculate cost
                c2_cost_prev = c2_cost
                c2_cost = SACSP_Eq_4_or_5(c2_data_avg_fourier, l, v, avg_cov_sum)
                c2_cost_increase = c2_cost - c2_cost_prev

            c2_filter_pairs.append([l, v])
    
    ## From the M x R pairs of spatial and spectral filters, select R pairs that maximize the cost function
    
    
    
    
    return c1_filter_pairs, c2_filter_pairs

In [ ]:
def SACSP2_Eq_4_or_5(data_fourier, h_or_l, w_or_v, avg_cov_sum):
    
    numerator = (w_or_v.T @ data_fourier @ np.diag(h_or_l) @ np.matrix(data_fourier).H @ w_or_v)[0, 0]
    denominator = (w_or_v.T @ avg_cov_sum @ w_or_v)[0, 0]
    
    return numerator / denominator

In [ ]:
def SACSP2_Eq_6_or_7(data_fourier, h_or_l, avg_cov_sum, top_n):
    E = data_fourier @ np.diag(h_or_l) @ np.matrix(data_fourier).H
    
    ## Generalized eigen-decomposition of Eq6 and Eq7 along with sum of class' spatial covariances
    eigval, eigvec = sp.linalg.eigh(E, avg_cov_sum)
    
    ## Extract top n eigen vectors
    sort_indices = np.argsort(eigval)
    top_n_indices = list(sort_indices[-top_n:])
    top_n_eigvec = eigvec[:, top_n_indices]
    
    return top_n_eigvec

In [ ]:
def SACSP2(c1_data, c2_data, R=3, M=1, e=1e-6):
    
    ## R = number of spectral/spatial filters for each class
    ## M = number of initializations of spectral filters
    
    t = c1_data.shape[-1]  ## Number of time samples
    F = sp.linalg.dft(t)  ## Fourier matrix with shape t x t
    
    H = np.ones((M, t))  ## Initialize M number of h vectors
    L = np.ones((M, t))  ## Initialize M number of l vectors
 
    c1_data_avg = np.mean(c1_data, axis=0)
    c2_data_avg = np.mean(c2_data, axis=0)

    c1_data_avg_cov = np.cov(c1_data_avg)
    c2_data_avg_cov = np.cov(c2_data_avg)
    avg_cov_sum = c1_data_avg_cov + c2_data_avg_cov
    
    c1_data_avg_fourier = c1_data_avg @ F
    c2_data_avg_fourier = c2_data_avg @ F
    
    c1_filter_pairs = []
    c2_filter_pairs = []
    
    for m in range(M):
        
        ## h and l are spectral filters
        h = H[m, :]
        l = L[m, :]
        
        ## w and v are spatial filters
        W = SACSP_Eq_6_or_7(c1_data_avg_fourier, h, avg_cov_sum, top_n=R)
        V = SACSP_Eq_6_or_7(c2_data_avg_fourier, l, avg_cov_sum, top_n=R)
        
        for r in range(R):  
            
            ### Class 1 optimization ###
            
            w = np.atleast_2d(W[:, r]).T  ## From W, get w as a column vector
            
            c1_cost = 0
            c1_cost_increase = e + 1
            
            while c1_cost_increase > e:
            
                ## Update spectral filter h_r_m from Eq9    
                E = np.matrix(c1_data_avg_fourier).H @ w @ w.T @ c1_data_avg_fourier
                
                for k in range(t):
                    h[k] = E[k, k] / np.linalg.norm(np.diag(E))
                
                H[m, :] = h
                
                ## Update spatial filter w_r by selecting the eigenvector 
                ##   corresponding to the largest eigenvalue from Eq6
                w = SACSP_Eq_6_or_7(c1_data_avg_fourier, h, avg_cov_sum, top_n=1)
                W[:, r] = w.flatten()

                ## Calculate cost
                c1_cost_prev = c1_cost
                c1_cost = SACSP_Eq_4_or_5(c1_data_avg_fourier, h, w, avg_cov_sum)
                c1_cost_increase = c1_cost - c1_cost_prev
   
            c1_filter_pairs.append([h, w])
            
            
            ### Class 2 optimization ###
            
            v = np.atleast_2d(V[:, r]).T  ## From V, get v as a column vector
            
            c2_cost = 0
            c2_cost_increase = e + 1
            
            while c2_cost_increase > e:
            
                ## Update spectral filter l_r_m from Eq10    
                E = np.matrix(c2_data_avg_fourier).H @ v @ v.T @ c2_data_avg_fourier
                
                for k in range(t):
                    l[k] = E[k, k] / np.linalg.norm(np.diag(E))
                
                L[m, :] = l
                
                ## Update spatial filter v_r by selecting the eigenvector 
                ##   corresponding to the largest eigenvalue from Eq7
                v = SACSP_Eq_6_or_7(c2_data_avg_fourier, l, avg_cov_sum, top_n=1)
                V[:, r] = v.flatten()

                ## Calculate cost
                c2_cost_prev = c2_cost
                c2_cost = SACSP_Eq_4_or_5(c2_data_avg_fourier, l, v, avg_cov_sum)
                c2_cost_increase = c2_cost - c2_cost_prev

            c2_filter_pairs.append([l, v])
    
    ## From the M x R pairs of spatial and spectral filters, select R pairs that maximize the cost function
    
    
    
    
    return c1_filter_pairs, c2_filter_pairs

In [ ]:
bc4_2a_s1_data_c1_filter_pairs, bc4_2a_s1_data_c2_filter_pairs = SACSP(bc4_2a_s1_data_c1, bc4_2a_s1_data_c2)

In [ ]:
np.diag(bc4_2a_s1_data_c1_filter_pairs[0][1][:, 0]).shape

In [ ]:
bc4_2a_s1_data_c1_filter_pairs[0][0]

In [ ]:
def SACSP_extract_features(all_data, c1_filter_pairs, c2_filter_pairs, R=3):
    
    t = all_data.shape[-1]  ## Number of time samples
    F = sp.linalg.dft(t)  ## Fourier matrix with shape t x t
    
    extracted_features = np.zeros((all_data.shape[0], 2 * R))
    
    for i, epoch in enumerate(all_data):    
        epoch_fourier = epoch @ F
        
        for j, filter_pair in enumerate(c1_filter_pairs + c2_filter_pairs):
            spectral_filter = filter_pair[0]  
            spatial_filter = filter_pair[1]
            extracted_feature = np.log(spatial_filter.T @ 
                                       epoch_fourier @ 
                                       np.diag(spectral_filter) @ 
                                       np.matrix(epoch_fourier).H @ 
                                       spatial_filter)
            
            extracted_features[i, j] = extracted_feature[0, 0]
            
    return extracted_features

In [ ]:
extracted_features = SACSP_extract_features(bc4_2a_s1_data, 
                                            bc4_2a_s1_data_c1_filter_pairs, 
                                            bc4_2a_s1_data_c2_filter_pairs)

In [ ]:
extracted_features[0]

In [ ]:
h = np.arange(5)

In [ ]:
np.atleast_2d(h).T.shape

In [ ]:
np.diag(h)

In [ ]:
np.diag(np.diag(h))

In [ ]:
a = 1
b = 2

c = []
c.append([a, b])

a += 1

print(c)